In [38]:
import os
import pandas as pd
pd.options.display.max_rows = 10
import matplotlib.pyplot as plt
import numpy as np
import datetime
from datetime import timedelta
import time

cantonKeys = ['AG','AI','AR', 'BE', 'BL', 'BS', 'FR', 'GE', 'GL', 'GR', 'JU', 'LU', 'NE', 'NW', 'OW', 'SG', 'SH', 'SO', 'SZ', 'TG', 'TI', 'UR', 'VD', 'VS', 'ZG','ZH']

# we discard the first days of 2020 and future data
yesterday = str(datetime.date.today()- timedelta(days = 1))
start = '2020-02-15'
end = '2021-04-05' #use '2021-04-05' for testing and yesterday for production

mergedDict = {}
for cantonId in cantonKeys: 
    mergedDict[cantonId] = pd.read_csv("data/merged/"+cantonId+".csv").set_index('date')[start:end]
    mergedDict[cantonId].index = pd.to_datetime(mergedDict[cantonId].index)
    
interpolMet = 'linear'
originalDict = mergedDict.copy()

In [39]:

for cantonId in cantonKeys:
    merged = mergedDict[cantonId].copy()
    
    # fill missing vaccine data
    vaccine = ['VaccDosesAdministered sumTotal','VaccDosesAdministered per100PersonsTotal', 
               'FullyVaccPersons sumTotal', 'FullyVaccPersons per100PersonsTotal']
    merged.loc['2020-02-15',vaccine] = 0
    merged[vaccine] = merged[vaccine].fillna(method='ffill')
    
    # fill missing total hospital capacities
    hospitalCols = ['ICU_Capacity','ICU_FreeCapacity','Total_Capacity','Total_FreeCapacity']
    merged[hospitalCols] = merged[hospitalCols].interpolate(method=interpolMet)
    merged[hospitalCols] = merged[hospitalCols].fillna(method='bfill')  

    # fill in missing hospital capacity data
    # we make the assumption that at the beginning there were no covid patients
    merged.loc['2020-02-15',['Total_Covid19Patients','ICU_Covid19Patients']] = 0
    patientCols = ['Total_Covid19Patients','Total_NonCovid19Patients','Total_AllPatients','ICU_Covid19Patients',
                   'ICU_NonCovid19Patients','ICU_AllPatients']
    merged[patientCols] = merged[patientCols].interpolate(method=interpolMet)
    merged[['Total_NonCovid19Patients','ICU_NonCovid19Patients']] = merged[['Total_NonCovid19Patients','ICU_NonCovid19Patients']].fillna(method='bfill')
    # Covid19Patients + NonCovid19Patients = AllPatients
    merged['Total_AllPatients'].fillna(merged[['Total_Covid19Patients','Total_NonCovid19Patients']].sum(axis=1), inplace=True)
    merged['ICU_AllPatients'].fillna(merged[['ICU_Covid19Patients','ICU_NonCovid19Patients']].sum(axis=1), inplace=True)
    
    # fill in missing Google mobility data
    googleMobilityCols = ['retail_and_recreation_percent_change_from_baseline','grocery_and_pharmacy_percent_change_from_baseline',
    'parks_percent_change_from_baseline','transit_stations_percent_change_from_baseline','workplaces_percent_change_from_baseline'
    ,'residential_percent_change_from_baseline']
    # use the swiss average when possible
    googleMobCH = pd.read_csv("data/GoogleMobility/2020_CH_Region_Mobility_Report.csv")
    googleMobCH = googleMobCH.loc[googleMobCH["sub_region_1"].isna()].set_index('date')[googleMobilityCols]
    googleMobCH.index = pd.to_datetime(googleMobCH.index)
    for col in googleMobilityCols:
        merged[col].fillna(googleMobCH[col], inplace=True)
    # for the rest interpolate
    merged[googleMobilityCols] = merged[googleMobilityCols].interpolate(method=interpolMet)
    merged[googleMobilityCols] = merged[googleMobilityCols].fillna(method='ffill')
    
    # fill in missing Intervista mobility data 
    merged[['intervistaMob']] = merged[['intervistaMob']].interpolate(method=interpolMet)
    merged['intervistaMob'].fillna(method='ffill', inplace=True)
        
    # fill in missing r values
    rvalues = ['median_R_mean','median_R_highHPD','median_R_lowHPD']
    merged[rvalues] = merged[rvalues].interpolate(method=interpolMet)  
    merged[rvalues] = merged[rvalues].fillna(method='ffill')
    merged[rvalues] = merged[rvalues].fillna(method='bfill')
  
    # fill in missing variants
    # first detected case for variants of concerne in Switzerland is 2020-10-14
    variants = ['lower_ci_day','upper_ci_day','anteil_pos']
    merged.loc['2020-10-13',['lower_ci_day','anteil_pos']] = 0
    merged.loc['2020-10-13',['upper_ci_day']] = 100
    merged[variants] = merged[variants].interpolate(method=interpolMet)
    merged[variants] = merged[variants].fillna(method='ffill')
    merged[variants] = merged[variants].fillna(method='bfill')

    # fill in daily incoming missing data
    zeroAndInterpolate = ['case_entries','hosp_entries','death_entries','case_inz_entries','hosp_inz_entries',
                          'death_inz_entries','case_inzsumTotal','hosp_inzsumTotal','death_inzsumTotal',
                          'test_inzsumTotal', 'meanNeighborIncidence', 'maxNeighborIncidence',
                          'Cases entries 0 - 9','Cases entries 10 - 19','Cases entries 20 - 29',
                          'Cases entries 30 - 39','Cases entries 40 - 49','Cases entries 50 - 59',
                          'Cases entries 60 - 69','Cases entries 70 - 79','Cases entries 80+',
                          'Cases entries male','Cases entries female','Cases inz_entries 0 - 9',
                          'Cases inz_entries 10 - 19','Cases inz_entries 20 - 29','Cases inz_entries 30 - 39',
                          'Cases inz_entries 40 - 49','Cases inz_entries 50 - 59','Cases inz_entries 60 - 69',
                          'Cases inz_entries 70 - 79','Cases inz_entries 80+','Cases inz_entries male',
                          'Cases inz_entries female','Cases inzsumTotal 0 - 9','Cases inzsumTotal 10 - 19',
                          'Cases inzsumTotal 20 - 29','Cases inzsumTotal 30 - 39','Cases inzsumTotal 40 - 49',
                          'Cases inzsumTotal 50 - 59','Cases inzsumTotal 60 - 69','Cases inzsumTotal 70 - 79',
                          'Cases inzsumTotal 80+','Cases inzsumTotal male','Cases inzsumTotal female',
                          'Death entries 0 - 9','Death entries 10 - 19','Death entries 20 - 29',
                          'Death entries 30 - 39','Death entries 40 - 49','Death entries 50 - 59',
                          'Death entries 60 - 69','Death entries 70 - 79','Death entries 80+',
                          'Death entries male','Death entries female','Death inz_entries 0 - 9',
                          'Death inz_entries 10 - 19','Death inz_entries 20 - 29','Death inz_entries 30 - 39',
                          'Death inz_entries 40 - 49','Death inz_entries 50 - 59','Death inz_entries 60 - 69',
                          'Death inz_entries 70 - 79','Death inz_entries 80+','Death inz_entries male',
                          'Death inz_entries female','Death inzsumTotal 0 - 9','Death inzsumTotal 10 - 19',
                          'Death inzsumTotal 20 - 29','Death inzsumTotal 30 - 39','Death inzsumTotal 40 - 49',
                          'Death inzsumTotal 50 - 59','Death inzsumTotal 60 - 69','Death inzsumTotal 70 - 79',
                          'Death inzsumTotal 80+','Death inzsumTotal male','Death inzsumTotal female',
                          'Hosp entries 0 - 9','Hosp entries 10 - 19','Hosp entries 20 - 29',
                          'Hosp entries 30 - 39','Hosp entries 40 - 49','Hosp entries 50 - 59',
                          'Hosp entries 60 - 69','Hosp entries 70 - 79','Hosp entries 80+','Hosp entries male',
                          'Hosp entries female','Hosp inz_entries 0 - 9','Hosp inz_entries 10 - 19',
                          'Hosp inz_entries 20 - 29','Hosp inz_entries 30 - 39','Hosp inz_entries 40 - 49',
                          'Hosp inz_entries 50 - 59','Hosp inz_entries 60 - 69','Hosp inz_entries 70 - 79',
                          'Hosp inz_entries 80+','Hosp inz_entries male','Hosp inz_entries female',
                          'Hosp inzsumTotal 0 - 9','Hosp inzsumTotal 10 - 19','Hosp inzsumTotal 20 - 29',
                          'Hosp inzsumTotal 30 - 39','Hosp inzsumTotal 40 - 49','Hosp inzsumTotal 50 - 59',
                          'Hosp inzsumTotal 60 - 69','Hosp inzsumTotal 70 - 79','Hosp inzsumTotal 80+',
                          'Hosp inzsumTotal male','Hosp inzsumTotal female']
    merged.loc['2020-02-15',zeroAndInterpolate] = 0
    merged[zeroAndInterpolate] = merged[zeroAndInterpolate].interpolate(method=interpolMet)
    
    if not os.path.exists('data/filled'):
        os.makedirs('data/filled')
    merged.to_csv('data/filled/'+cantonId+'.csv')

In [40]:
# check complete data if there are any NaNs left
for cantonId in cantonKeys:
    filled = pd.read_csv('data/filled/'+cantonId+'.csv')
    for col in filled.columns:
        if filled[col].isna().sum() != 0:
            print(cantonId+" "+col+" (#NaN/#NotNaN): (" + str(filled[col].isna().sum())+"/"+str(filled[col].notna().sum())+")")
            #dict[cantonId][col].plot(kind='line',y=[col], figsize=(20,10))
            #plt.show()

In [41]:
# plotting original data vs filled data
#for col in dict['AG'].columns:
#    comparingDf = pd.concat([dict['AG'][[col]],originalData[["original_"+col]]], axis=1)
    #comparingDf[['original_'+col]].reset_index().plot(kind='scatter', x=['date'], y=['original_'+col], figsize=(20,10))
    #comparingDf['2020-02-15':'2021-04-05'].plot(kind='line',y=[col], figsize=(20,10))


In [42]:
# FEATURE ENGINEERING
filledDict = {}

for cantonId in cantonKeys:
    filledDict[cantonId] = pd.read_csv("data/filled/"+cantonId+".csv").set_index('date')
    
    dailyFeatures = filledDict[cantonId].copy()

    # summarize mask mandatories
    maskMandatories = [ 'Mask mandatory in publicly accessible establishments/ spaces (shops etc.)',
                       'Mask mandatory in public transport','Masks mandatory in schools','Masks mandatory at work']
    dailyFeatures[['maskMandatories']] = dailyFeatures[maskMandatories].sum(axis=1)
    dailyFeatures.drop(maskMandatories, axis=1, inplace=True)

    dailyFeatures[['googleMobility']] = dailyFeatures[['retail_and_recreation_percent_change_from_baseline',
                                                       'grocery_and_pharmacy_percent_change_from_baseline',
                                                       'parks_percent_change_from_baseline',
                                                       'transit_stations_percent_change_from_baseline',
                                                       'workplaces_percent_change_from_baseline',
                                                       'residential_percent_change_from_baseline']].mean(axis=1)

    # r value accuracy
    dailyFeatures[['R_error']] = dailyFeatures['median_R_highHPD']-dailyFeatures['median_R_lowHPD']
    dailyFeatures.drop(['median_R_highHPD','median_R_lowHPD'],axis=1, inplace=True)

    # variants accuracy
    #features[['anteil_pos','upper_ci_day','lower_ci_day']] = features[['anteil_pos','upper_ci_day','lower_ci_day']].rolling(window=w).mean()
    dailyFeatures[['variant_error']] = dailyFeatures['upper_ci_day']-dailyFeatures['lower_ci_day']
    dailyFeatures.drop(['upper_ci_day','lower_ci_day'],axis=1, inplace=True)

    # vaccine
    dailyFeatures.drop(['VaccDosesAdministered sumTotal','FullyVaccPersons sumTotal'],axis=1, inplace=True)
    vaccine = ['VaccDosesAdministered per100PersonsTotal',
               'FullyVaccPersons per100PersonsTotal']
    #features[vaccine] = features[vaccine].rolling(window=w).mean()
    
    # test positivity rate
    #display(len(features[['case_entries']]))
    #display(len(features[['test_entries']]))
    dailyFeatures[['testPositvity']] = dailyFeatures['case_entries']/dailyFeatures['test_entries']

    # remove absolut values which are included in the incidenc rates
    absVal = ['Cases entries 0 - 9','Cases entries 10 - 19','Cases entries 20 - 29','Cases entries 30 - 39',
              'Cases entries 40 - 49','Cases entries 50 - 59','Cases entries 60 - 69','Cases entries 70 - 79',
              'Cases entries 80+','Death entries 0 - 9','Death entries 10 - 19','Death entries 20 - 29',
              'Death entries 30 - 39','Death entries 40 - 49','Death entries 50 - 59','Death entries 60 - 69',
              'Death entries 70 - 79','Death entries 80+', 'Hosp entries 0 - 9','Hosp entries 10 - 19',
              'Hosp entries 20 - 29','Hosp entries 30 - 39','Hosp entries 40 - 49','Hosp entries 50 - 59',
              'Hosp entries 60 - 69','Hosp entries 70 - 79','Hosp entries 80+', 'Cases entries female',
              'Cases entries male','Death entries female','Death entries male', 'Hosp entries female',
              'Hosp entries male','case_entries','hosp_entries','death_entries','test_entries']
    dailyFeatures.drop(absVal,axis=1, inplace=True)

    # hospital capacities
    hospCap = [ 'ICU_AllPatients',
     'ICU_Covid19Patients',
     'ICU_Capacity',
     'Total_AllPatients',
     'Total_Covid19Patients',
     'Total_Capacity',
     'ICU_NonCovid19Patients',
     'ICU_FreeCapacity',
     'Total_NonCovid19Patients',
     'Total_FreeCapacity']
    staticCantonal = pd.read_excel("static_data/staticCantonalData.xlsx").set_index('canton').transpose()
    dailyFeatures[[col + "_inz" for col in hospCap]] = 100000*(dailyFeatures[hospCap]/staticCantonal.loc[[cantonId]]['residents'][0])
    dailyFeatures.drop(hospCap,axis=1, inplace=True)

    if not os.path.exists('data/dailyFeatures'):
        os.makedirs('data/dailyFeatures')
    dailyFeatures.to_csv('data/dailyFeatures/'+cantonId+'.csv')

In [43]:
# CONSTRUCTING INPUT/OUTPUT INTERVALS


# generate date ranges
dataStart = pd.Timestamp(start)
dataEnd = pd.Timestamp(end)

daysIn = 7 # last n days of input
daysOut = 7 # next n days of output

listOfInputIntervals = []
listOfOutputIntervals = []
for e in pd.date_range(start=dataStart,end=dataEnd, freq='D'):
    if (e+timedelta(days = daysIn+daysOut-1) <= dataEnd.date()):
        listOfInputIntervals.append((e.date(),(e+timedelta(days = (daysIn-1))).date()))
        listOfOutputIntervals.append(((e+timedelta(days = daysIn)).date(),(e+timedelta(days = daysIn+daysOut-1)).date()))
    
#display(listOfInputIntervals)
#display(listOfOutputIntervals)

In [44]:
dailyFeaturesDict = {}
    
# CONSTRUCTING ACTUAL INPUTS
for cantonId in cantonKeys:
    dailyFeaturesDict[cantonId] = pd.read_csv("data/dailyFeatures/"+cantonId+".csv").set_index('date')
    dailyFeaturesDict[cantonId].index = pd.to_datetime(dailyFeaturesDict[cantonId].index)
    
    display(cantonId)
    features = pd.DataFrame()
    
    # construction of input features
    for t in listOfInputIntervals:
        
        # average features
        # features which will be averaged over the whole input interval
        averageFeatures = ['Cases inz_entries 0 - 9','Cases inz_entries 10 - 19','Cases inz_entries 20 - 29',
                  'Cases inz_entries 30 - 39','Cases inz_entries 40 - 49','Cases inz_entries 50 - 59',
                  'Cases inz_entries 60 - 69','Cases inz_entries 70 - 79','Cases inz_entries 80+',
                  'Cases inzsumTotal 0 - 9','Cases inzsumTotal 10 - 19','Cases inzsumTotal 20 - 29',
                  'Cases inzsumTotal 30 - 39','Cases inzsumTotal 40 - 49','Cases inzsumTotal 50 - 59',
                  'Cases inzsumTotal 60 - 69','Cases inzsumTotal 70 - 79','Cases inzsumTotal 80+',
                  'Death inz_entries 0 - 9','Death inz_entries 10 - 19','Death inz_entries 20 - 29',
                  'Death inz_entries 30 - 39','Death inz_entries 40 - 49','Death inz_entries 50 - 59',
                  'Death inz_entries 60 - 69','Death inz_entries 70 - 79','Death inz_entries 80+',
                  'Death inzsumTotal 0 - 9','Death inzsumTotal 10 - 19','Death inzsumTotal 20 - 29',
                  'Death inzsumTotal 30 - 39','Death inzsumTotal 40 - 49','Death inzsumTotal 50 - 59',
                  'Death inzsumTotal 60 - 69','Death inzsumTotal 70 - 79','Death inzsumTotal 80+',
                  'Hosp inz_entries 0 - 9','Hosp inz_entries 10 - 19','Hosp inz_entries 20 - 29',
                  'Hosp inz_entries 30 - 39','Hosp inz_entries 40 - 49','Hosp inz_entries 50 - 59',
                  'Hosp inz_entries 60 - 69','Hosp inz_entries 70 - 79','Hosp inz_entries 80+',
                  'Hosp inzsumTotal 0 - 9','Hosp inzsumTotal 10 - 19','Hosp inzsumTotal 20 - 29',
                  'Hosp inzsumTotal 30 - 39','Hosp inzsumTotal 40 - 49','Hosp inzsumTotal 50 - 59',
                  'Hosp inzsumTotal 60 - 69','Hosp inzsumTotal 70 - 79','Hosp inzsumTotal 80+',
                  'Cases inz_entries female','Cases inz_entries male','Cases inzsumTotal female',
                  'Cases inzsumTotal male','Death inz_entries female','Death inz_entries male',
                  'Death inzsumTotal female','Death inzsumTotal male','Hosp inz_entries female',
                  'Hosp inz_entries male','Hosp inzsumTotal female','Hosp inzsumTotal male', 
                  'VaccDosesAdministered per100PersonsTotal',
                  'FullyVaccPersons per100PersonsTotal',
                  'anteil_pos',
                  'variant_error',
                  'case_inzsumTotal','hosp_inzsumTotal','death_inzsumTotal','test_inzsumTotal','case_inz_entries',
                  'hosp_inz_entries','death_inz_entries','test_inz_entries','testPositvity',
                  'median_R_mean','R_error',
                  'meanNeighborIncidence','maxNeighborIncidence',
                  'kofStrigency',
                  'Borders','Events','Gatherings/private events','Demonstrations',
                  'Primary (includes kindergarten) and lower secondary school','Upper secondary school, vocational schools and higher education',
                  'universities and other educational establishments\xa0','Mountain railways','Homeworking','Restaurants',
                  'Discos/Nightclubs','Shops/Markets','Penalties','Cultural, entertainment and recreational facilities',
                  'Sport/Wellness facilities','Sport activities','Religious services','Singing allowed','maskMandatories',
                  'ICU_AllPatients_inz','ICU_Covid19Patients_inz','ICU_Capacity_inz','Total_AllPatients_inz',
                  'Total_Covid19Patients_inz','Total_Capacity_inz','ICU_NonCovid19Patients_inz','ICU_FreeCapacity_inz',
                  'Total_NonCovid19Patients_inz','Total_FreeCapacity_inz']
        featureRow = dailyFeaturesDict[cantonId][t[0]:t[1]][averageFeatures].mean().to_frame().transpose()

        # direct features
        # features which will be direct input for every day of the input interval
        # attention: this can potentially increase the number of input features significantly
        # added features are len(directFea)*daysIn
        # only add features for which have a large variance from one day to another day
        directFeatures = ['retail_and_recreation_percent_change_from_baseline',
                     'grocery_and_pharmacy_percent_change_from_baseline',
                     'parks_percent_change_from_baseline',
                     'transit_stations_percent_change_from_baseline',
                     'workplaces_percent_change_from_baseline',
                     'residential_percent_change_from_baseline',
                     'intervistaMob',
                     'isHolyday',
                     'temp_min','temp_max','clouds','precipitation']
        for f in directFeatures:
            directFe = dailyFeaturesDict[cantonId][t[0]:t[1]][[f]].transpose()
            directFe.columns = [f+'_day_'+str(d) for d in range(0,daysIn)]
            directFe = directFe.reset_index().drop(['index'], axis=1)
            featureRow = pd.concat([featureRow,directFe], axis=1)
        
        
        # future features
        futureFeatures = ['temp_min','temp_max','clouds','precipitation']
        indexOfInputTuple = listOfInputIntervals.index(t)
        ot = listOfOutputIntervals[indexOfInputTuple]
        for ff in futureFeatures:
            futureFe = dailyFeaturesDict[cantonId][ot[0]:ot[1]][[ff]].transpose()
            futureFe.columns = [ff+'_future_day_'+str(d) for d in range(0,daysOut)]
            futureFe = futureFe.reset_index().drop(['index'], axis=1)
            featureRow = pd.concat([featureRow,futureFe], axis=1)
        
        features = features.append(featureRow, ignore_index=True)

    # static cantonal features
    staticCantonal = pd.read_excel("static_data/staticCantonalData.xlsx").set_index('canton').transpose()
    # households
    households = ['1PersonHouseholds', '2PersonHouseholds','3PersonHouseholds', '4PersonHouseholds', 
                  '5PersonHouseholds','6+PersonHouseholds']
    for h in households:
        features[[h+"_perc"]] = staticCantonal.loc[[cantonId]][h][0]/staticCantonal.loc[[cantonId]]['totalHousholds'][0]
    features[['averageHousehold']] = staticCantonal.loc[[cantonId]]['residents'][0]/staticCantonal.loc[[cantonId]]['totalHousholds'][0]
    # add static features
    staticFeatures = ['percentage 65 years or over','urbanPopulationPercent','homeownershipPercent', 
                      'livingSpaceInm2','carsPer1000inhabitants', 'publicTransportationPercent',
                      'privateMotorisedTransportPercent','DoctorsPer100Kinhabitants','residentsPerKm2']
    for f in staticFeatures:
        features[[f]] = staticCantonal.loc[[cantonId]][f][0]
    # construct settlement area feature
    residents = staticCantonal.loc[[cantonId]]['residents'][0]
    settlementArea = staticCantonal.loc[[cantonId]]['areaInKm2'][0]*(staticCantonal.loc[[cantonId]]['settlementAreaPercent'][0]/100)
    features[['residentsPerKm2SettlementArea']] = residents/settlementArea    
    
    if not os.path.exists('data/features'):
        os.makedirs('data/features')
    features.to_csv('data/features/'+cantonId+'.csv', index=False)



# CONSTRUCTING ACTUAL OUTPUTS
for cantonId in cantonKeys:
    outputs = pd.DataFrame()
    for t in listOfOutputIntervals:
        outputCols = ['hosp_inz_entries','death_inz_entries','testPositvity','googleMobility']
        outputRow = dailyFeaturesDict[cantonId][t[0]:t[1]][outputCols].mean().to_frame().transpose()   
        outputs = outputs.append(outputRow, ignore_index=True)    
    if not os.path.exists('data/outputs'):
        os.makedirs('data/outputs')
    outputs.to_csv('data/outputs/'+cantonId+'.csv', index=False)


# MERGE ALL CANTONAL TRAIN DATA TOGETHER
train_features = pd.DataFrame()
train_labels = pd.DataFrame()
for cantonId in [canton for canton in cantonKeys if canton not in ['SG','NE','NW']]: # exclude test cantons
    train_features = train_features.append(pd.read_csv("data/features/"+cantonId+".csv"))
    train_labels = train_labels.append(pd.read_csv("data/outputs/"+cantonId+".csv"))
train_features.to_csv('train_features.csv', index=False)
train_labels.to_csv('train_labels.csv', index=False)
  
    
# MERGE ALL CANTONAL TEST DATA TOGETHER
test_features = pd.DataFrame()
test_labels = pd.DataFrame()
for cantonId in ['SG','NE','NW']:
    test_features = test_features.append(pd.read_csv("data/features/"+cantonId+".csv"))
    test_labels = test_labels.append(pd.read_csv("data/outputs/"+cantonId+".csv"))
test_features.to_csv('test_features.csv', index=False)
test_labels.to_csv('test_labels.csv', index=False)

'AG'

'AI'

'AR'

'BE'

'BL'

...

'UR'

'VD'

'VS'

'ZG'

'ZH'